In [188]:
import copy
import multiprocessing as mp
import multiprocessing.dummy as dummy_mp
import os
import re
from datetime import datetime
from string import Template

import pymorphy2
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

FB_LOGIN = '89026051080'
FB_PASSWORD = 'z6A-8Wd-XiV-VNY'
login, password = FB_LOGIN, FB_PASSWORD

__base_url = 'https://www.facebook.com'
__base_group_url = 'https://www.facebook.com/groups'
__login_url = 'https://www.facebook.com/login'

options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")

options.add_argument("--disable-software-rasterizer")  # maybe not needed
options.add_argument("--disable-dev-shm-usage")  # maybe not needed

browser = webdriver.Chrome(options = options)

In [189]:
def enter_facebook(browser, login, password):
    """Login to fb

    :param login: (str) fb login
    :param password: (str) fb password

    :return: -1 if error
    """
    browser.get(__login_url)

    try:
        login_form = browser.find_element_by_xpath("//input[@id='email' or @name='email']")
        login_form.clear()
        login_form.send_keys(login)
    except Exception as e:
        print(e)
        print('error: email')
        return -1

    try:
        pass_form = browser.find_element_by_xpath("//input[@id='pass' or @name='pass']")
        pass_form.clear()
        pass_form.send_keys(password)
    except Exception as e:
        print(e)
        print('error: password')
        return -1

    try:
        browser.find_element_by_xpath("//button[@id='loginbutton']").click()
    except Exception as e:
        print(e)
        print('error: login button')
        return -1

    if 'login_attempt' in browser.current_url:
        return -1

In [190]:
enter_facebook(browser, FB_LOGIN, FB_PASSWORD)

In [12]:
# posts = fb.get_posts('100032357613324')
page_id = '100032357613324'
browser.get(__base_url + '/' + page_id)

page_soup = BeautifulSoup(browser.page_source, 'lxml')

In [15]:
nonavbl = page_soup.find('title', text=re.compile('.*Страница не найдена.*'))
if nonavbl:
    print('Страница не найдена')
nonavbl = page_soup.find('h2', text=re.compile('.*Эта страница недоступна.*'))
if nonavbl:
    print('Эта страница недоступна')
nonavbl = page_soup.find('h2', text=re.compile('.*К сожалению, этот контент сейчас недоступен.*'))
if nonavbl:
    print('К сожалению, этот контент сейчас недоступен')

In [16]:
page_soup.find('title', text=re.compile('.*Страница не найдена.*'))

### Блок для замены функции поиска постов

In [250]:
from time import sleep 
from datetime import datetime, timedelta

from selenium.webdriver.common.action_chains import ActionChains
actions = ActionChains(browser)
limit = 20

In [321]:
date1 = '24 марта в 10:20'
date2 = '1 ч.'
date3 = '11 февраля'
date4 = '2 д.'
date5 = '30 мин.'

textToDate_months = {
    'января': 1,
    'февраля': 2,
    'марта': 3,
    'фпреля': 4,
    'мая': 5,
    'июня': 6,
    'июля': 7,
    'августа': 8,
    'сентября': 9,
    'октября': 10,
    'ноября': 11,
    'декабря': 12
}

def textToDate(text):        
    date = datetime.today()
    if text:  
        s = text.split()
        t = int(s[0])
        if 'ч.' in s[1]:
            date -= timedelta(hours=t)
        elif 'д.' in s[1]:
            date -= timedelta(days=t)
        elif 'мин.' in s[1]:
            date -= timedelta(minutes=t)
        else:
            date = datetime(date.year, int(textToDate_months.get(s[1], date.month)), t)                
    return date
    

[textToDate(d) for d in [date1, date2, date3, date4, date5]]

[datetime.datetime(2021, 3, 24, 0, 0),
 datetime.datetime(2021, 4, 10, 18, 33, 5, 628457),
 datetime.datetime(2021, 2, 11, 0, 0),
 datetime.datetime(2021, 4, 8, 19, 33, 5, 628457),
 datetime.datetime(2021, 4, 10, 19, 3, 5, 628457)]

In [317]:
def getPostDate(browser, post):
    browser.execute_script("arguments[0].scrollIntoView();", post)
    abilities = post.find_elements_by_xpath('.//span/a/span')
    if len(abilities)> 0:
        return textToDate(abilities[0].text)
    return datetime.today()            

In [197]:
# try:
#     WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.TAG_NAME, "div[role='article']")))
# except TimeoutException:
#     print(posts_list) 

In [339]:
from_date = datetime.today() - timedelta(days=2)

In [348]:
postcount = -1
posts_list = []
error_count = 0
mindate = datetime.today()

while ((not from_date and len(posts_list) < limit) or (from_date and mindate >= from_date)) and len(posts_list) != postcount:
    postcount = len(posts_list)
    for i in range(3):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(0.5)
    posts_list = browser.find_elements_by_tag_name("div[role='article'][aria-posinset]")
    if len(posts_list)>0:
        mindate = getPostDate(browser, posts_list[-1])
    print(len(posts_list), postcount)

posts_list = [post for post in posts_list if getPostDate(browser, post) >= from_date]
print(len(posts_list), postcount)

80 0
83 80
86 83
89 86
92 89
95 92
98 95
101 98
104 101
107 104
110 107
113 110
116 113
113 113


In [347]:
 from_date and mindate >= from_date

True

In [312]:
post = posts_list[7]

In [260]:
more = post.find_elements_by_xpath('.//div[contains(text(), "Ещё")]')
if more:
    more[0].click()
    
more_comments_finder = lambda: post.find_elements_by_xpath('.//span[starts-with(text(), "Посмотреть ещё")]') + post.find_elements_by_xpath('.//span[starts-with(text(), "Показать ещё")]')
more_comments = more_comments_finder()
while more_comments:
    more_comments[0].click()
    sleep(1)
    more_comments = more_comments_finder()    
    
comments = post.find_elements_by_tag_name("ul")[1]

In [341]:
print(len(posts_list))
dates = [getPostDate(browser, post) for post in posts_list]
print(len(dates))
dates

29
29


[datetime.datetime(2021, 4, 10, 19, 55, 51, 558392),
 datetime.datetime(2021, 4, 10, 19, 52, 51, 652348),
 datetime.datetime(2021, 4, 10, 19, 21, 51, 728294),
 datetime.datetime(2021, 4, 10, 19, 9, 51, 783822),
 datetime.datetime(2021, 4, 10, 17, 9, 51, 861778),
 datetime.datetime(2021, 4, 10, 16, 9, 51, 934736),
 datetime.datetime(2021, 4, 10, 16, 9, 52, 40686),
 datetime.datetime(2021, 4, 10, 15, 9, 52, 132647),
 datetime.datetime(2021, 4, 10, 14, 9, 52, 220597),
 datetime.datetime(2021, 4, 10, 14, 9, 52, 313541),
 datetime.datetime(2021, 4, 10, 13, 9, 52, 393495),
 datetime.datetime(2021, 4, 10, 12, 9, 52, 476709),
 datetime.datetime(2021, 4, 10, 12, 9, 52, 566659),
 datetime.datetime(2021, 4, 10, 11, 9, 52, 659623),
 datetime.datetime(2021, 4, 10, 10, 9, 52, 761529),
 datetime.datetime(2021, 4, 10, 10, 9, 52, 845480),
 datetime.datetime(2021, 4, 10, 9, 9, 52, 940442),
 datetime.datetime(2021, 4, 10, 8, 9, 53, 30393),
 datetime.datetime(2021, 4, 10, 7, 9, 53, 122332),
 datetime.date

In [234]:
post.get_attribute('innerHTML')

'<div class="j83agx80 cbu4d94t" style=""><div class="rq0escxv l9j0dhe7 du4w35lb"><div class="j83agx80 l9j0dhe7 k4urcfbm"><div class="rq0escxv l9j0dhe7 du4w35lb hybvsw6c io0zqebd m5lcvass fbipl8qg nwvqtn77 k4urcfbm ni8dbmo4 stjgntxs sbcfpzgs" style="border-radius: max(0px, min(8px, ((100vw - 4px) - 100%) * 9999)) / 8px;"><div><div></div><div><div><div></div><div><div class="pybr56ya dati1w0a hv4rvrfc n851cfcs btwxx1t3 j83agx80 ll8tlv6m"><div class="oi9244e8 do00u71z j83agx80"><div class="nc684nl6"><a aria-hidden="true" class="oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl oo9gr5id gpro0wi8" href="https://www.facebook.com/themeduza/?__cft__[0]=AZWgRp8yGuxab7xBmkaoHAkvE6SQd6_-7WVpzAw-WWJLmjnOKkQNvHBlcUJQvo3qaDwjB1aFjnCaB7Bq-ubBhcCLvO-gvPdBMLyg8xWsSsqAsmYHmXM3mcPwJ1a9r0lzgf4wDBDETFp64rQq2htclwtn&amp;__tn__=%3C%2CP-R" role="link" tabindex="-1